In [546]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk import PorterStemmer
from nltk import LancasterStemmer


In [547]:
if not nltk.download('stopwords'):
    nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jatin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [548]:
data = pd.read_csv('data/train.csv')
data.shape

(20800, 5)

In [549]:
data.dtypes

id         int64
title     object
author    object
text      object
label      int64
dtype: object

In [550]:
data.isna().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [551]:
data = data.fillna('')

In [552]:
data['article'] = data['title'] + data['author'] + data['text']

In [553]:
data.drop(columns=['id'], inplace=True)

In [554]:
data['article'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - BreitbartDaniel J. FlynnEver get the feeling your life circles the roundabout rather than heads in a straight line toward the intended destination? [Hillary Clinton remains the big woman on campus in leafy, liberal Wellesley, Massachusetts. Everywhere else votes her most likely to don her inauguration dress for the remainder of her days the way Miss Havisham forever wore that wedding dress.  Speaking of Great Expectations, Hillary Rodham overflowed with them 48 years ago when she first addressed a Wellesley graduating class. The president of the college informed those gathered in 1969 that the students needed “no debate so far as I could ascertain as to who their spokesman was to be” (kind of the like the Democratic primaries in 2016 minus the   terms unknown then even at a Seven Sisters school). “I am very glad that Miss Adams made it clear that what I am speaking for today is all of us —  the 400 of us,” Miss Rodham told her classmates. 

In [555]:
# X = data.drop(columns='label', axis=1)
# Y = data['label']

In [556]:
ps = PorterStemmer()
ls = LancasterStemmer()


In [557]:
# stemmed_word = re.sub('[`~!@#$%^&*()\\]\\[+={}/|:;\"\'<>,.?-_]', X)
def perform_stemming(content):
    remove_special_char = re.sub('[^a-zA-Z]',' ', content)
    string_to_lower = remove_special_char.lower().split()
    #string_split = string_to_lower.split()
    remove_word = [word for word in string_to_lower if not word in stopwords.words('english')]
    stem = [ps.stem(word) for word in remove_word]
    return stem

In [558]:
data['article'] = data['article'].apply(perform_stemming)

In [ ]:
def perform_stemming_test(data,col):
    data[col] = data[col].replace('[^a-zA-Z]',' ',regex=True)
    data[col] = data[col].applymap(lambda s:s.lower().split() if type(s) == str else s)
    # #data[col] = data[col].apply(lambda x:x.split())
    data[col]=data[col].apply(lambda x: [item for item in x if item not in stopwords.words('english')])
    data[col] = data[col].apply(lambda x: [ps.stem(y) for y in x])
    return data

In [ ]:
#data = perform_stemming_test(data,['article'])

AttributeError: 'list' object has no attribute 'lower'

In [ ]:
data['article']

0        [house, dem, aide, we, didn, t, even, see, com...
1        [flynn, hillary, clinton, big, woman, on, camp...
2        [why, the, truth, might, get, you, firedconsor...
3        [civilians, killed, in, single, us, airstrike,...
4        [iranian, woman, jailed, for, fictional, unpub...
                               ...                        
20795    [rapper, t, i, trump, a, poster, child, for, w...
20796    [n, f, l, playoffs, schedule, matchups, and, o...
20797    [macy, s, is, said, to, receive, takeover, app...
20798    [nato, russia, to, hold, parallel, exercises, ...
20799    [what, keeps, the, f, alivedavid, swanson, dav...
Name: article, Length: 20800, dtype: object